In [1]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

mynew2023


## Import Required Packages

In [14]:
import os
import matplotlib.pylab as plt
import numpy as np
import torch
import torchvision

### Load MEG Data

In [15]:
import os, glob
from distutils.dir_util import copy_tree, remove_tree
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import mne

In [26]:
from pathlib import Path

datapath = os.path.join(Path(os.getcwd()).parent.parent.parent, 'nshor', 'Multiband_with_MEG/')
# print(datapath)


### List of subjects

In [30]:
tmp_path = os.path.join(os.getcwd(), 'my_derivatives/')
list_sub =[os.path.basename(s) for s in glob.glob(datapath +'sub*')]
print(list_sub)

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12']


### Locating individual meg files

In [37]:
# for sub in tqdm(sorted(list_sub)):
#     #print(sub)

sub = 'sub-02'
# create and copy data to temproray directory
tmp_dir = os.path.join(tmp_path,sub)

#Inverse problem output directory
inverse_dir_output = os.path.join( tmp_path,sub,'meg','Inverse/')
path_in_source = os.path.join(datapath,sub,'meg', sub+'_task-RDR_run-1_meg.fif')
if os.path.exists(path_in_source):

    if not os.path.exists(inverse_dir_output):
        #making Inverse folder in tmp
        os.makedirs(inverse_dir_output)

In [39]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_raw.fif')
# print(sample_data_raw_file)
raw = mne.io.read_raw_fif(sample_data_raw_file)

Opening raw data file /home/users/mrahman21/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


In [40]:
n_time_samps = raw.n_times
time_secs = raw.times
ch_names = raw.ch_names
n_chan = len(ch_names)  # note: there is no raw.n_channels attribute
print('the (cropped) sample data object has {} time samples and {} channels.'
      ''.format(n_time_samps, n_chan))
print('The last time sample is at {} seconds.'.format(time_secs[-1]))
print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
print()  # insert a blank line in the output

# some examples of raw.info:
print('bad channels:', raw.info['bads'])  # chs marked "bad" during acquisition
print(raw.info['sfreq'], 'Hz')            # sampling frequency
print(raw.info['description'], '\n')      # miscellaneous acquisition info

print(raw.info)

the (cropped) sample data object has 166800 time samples and 376 channels.
The last time sample is at 277.7136813300495 seconds.
The first few channel names are MEG 0113, MEG 0112, MEG 0111.

bad channels: ['MEG 2443', 'EEG 053']
600.614990234375 Hz
acquisition (megacq) VectorView system at NMR-MGH 

<Info | 21 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 Gradiometers, 102 Magnetometers, 9 Stimulus, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nc

#### Compute sLORETA/dSPM inverse solution on raw dataset restricted to a brain label and stores the solution in stc files for visualisation.

In [41]:
import matplotlib.pyplot as plt

import mne
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_raw, read_inverse_operator

print(__doc__)

data_path = sample.data_path()
fname_inv = (
    data_path / 'MEG' / 'sample' / 'sample_audvis-meg-oct-6-meg-inv.fif')
fname_raw = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
label_name = 'Aud-lh'
fname_label = data_path / 'MEG' / 'sample' / 'labels' / f'{label_name}.label'

snr = 1.0  # use smaller SNR for raw data
lambda2 = 1.0 / snr ** 2
method = "MNE" #"dSPM" #"sLORETA"  # use sLORETA method (could also be MNE or dSPM)

# Load data
raw = mne.io.read_raw_fif(fname_raw)
inverse_operator = read_inverse_operator(fname_inv)
label = mne.read_label(fname_label)

raw.set_eeg_reference('average', projection=True)  # set average reference.
start, stop = raw.time_as_index([0, 15])  # read the first 15s of data

# Compute inverse solution
stc = apply_inverse_raw(raw, inverse_operator, lambda2, method, label,
                        start, stop, pick_ori=None)

print(type(stc))
# Save result in stc files
stc.save('mne_%s_raw_inverse_%s' % (method, label_name), overwrite=True)

Automatically created module for IPython interactive environment
Opening raw data file /home/users/mrahman21/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading inverse operator decomposition from /home/users/mrahman21/mne_data/MNE-sample-data/MEG/sample/sample_audvis-meg-oct-6-meg-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    305 x 305 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Noise covariance matrix read.
    22494 x 22494 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    22494 x 22494 diagonal c